# Loading Experiment Data

In this notebook, we start by loading the data collected while running different experiment-wares (in this case, COP solvers), and perform some preprocessing on this data to allow its use for further analysis in dedicated notebooks.

## Imports

We first need to import the modules we need to load the data.
In particular, we must obviously import *Metrics-Wallet*, which we will use to deal with our data.

In [1]:
from itertools import product
from metrics.wallet import BasicAnalysis
from pandas import isnull

## Reading the data

The next step is to read the data from the log files produced by our different experiment-wares.
This data is described in the file [`scalpel_config.yml`](config/scalpel_config.yml), and automatically parsed by *Metrics-Scalpel* to create a `BasicAnalysis` object.

In [2]:
analysis = BasicAnalysis(input_file='config/scalpel_config.yml', log_level='WARNING')

The `BasicAnalysis` object instantiated above provides elementary and general methods for preprocessing our data before actually analyzing the results (which will require more specific methods as it can be seen in the dedicated notebooks).

An important thing to do now is to visualize the collected data, to make sure that everything was properly read.
This can be achieved by looking at the data-frame that has been built inside the `BasicAnalysis` object.

In [3]:
analysis.data_frame

,input,experiment_ware,cpu_time,track,status,bound_list,timestamp_list,family,cop.way,timeout,success,user_success,missing,consistent_xp,consistent_input,error
0,Rcpsp_m1_j30_Rcpsp_j30_43_02,choco,1.326810,COP,OPTIMUM FOUND,"[44, 43]","[1.12, 1.12]",Rcpsp,NaN,1200.0,True,True,False,True,True,False
9745,Rcpsp_m1_j30_Rcpsp_j30_43_02,cosoco,0.033937,None,OPTIMUM,43,0.000442,Rcpsp,None,1200.0,True,True,False,True,True,False
5079,Rcpsp_m1_j30_Rcpsp_j30_43_02,ace,2.674830,None,OPTIMUM FOUND,"[55, 51, 50, 43]","[0.41, 0.43, 0.43, 0.44]",Rcpsp,min,1200.0,True,True,False,True,True,False
1377,Rcpsp_m1_j30_Rcpsp_j30_43_02,sat4j+default,240.661000,None,OPTIMUM FOUND,"[143, 131, 129, 116, 115, 108, 107, 106, 105, ...",None,Rcpsp,None,1200.0,True,True,False,True,True,False
16164,GraphColoring_m1_mono_GraphColoring_2_fullins_5,choco,300.085000,COP,SATISFIABLE,6,5.93,GraphColoring,None,1200.0,True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18249,PseudoBoolean_opt_factor_Pb_factor_S9_P191_Q389,sat4j+default,900.566000,None,UNKNOWN,None,None,PseudoBoolean,None,1200.0,True,True,False,True,True,False
18402,Rcpsp_m1_j60_Rcpsp_j60_14_07,choco,2.021590,COP,OPTIMUM FOUND,"[314, 313, 312, 72, 71, 69]","[1.76, 1.76, 1.76, 1.76, 1.76, 1.76]",Rcpsp,None,1200.0,True,True,False,True,True,False
18201,Rcpsp_m1_j60_Rcpsp_j60_14_07,cosoco,0.045985,None,OPTIMUM,69,0.003777,Rcpsp,None,1200.0,True,True,False,True,True,False
18582,Rcpsp_m1_j60_Rcpsp_j60_14_07,ace,7.964840,None,OPTIMUM FOUND,"[82, 79, 69]","[1.19, 1.26, 1.34]",Rcpsp,min,1200.0,True,True,False,True,True,False


## More meaningful names for experiment-wares

Currently, the names of the experiment-wares correspond to those extracted by *Metrics-Scalpel*.
While they are sufficient to discriminate them in the analysis, they are not necessarily meaningful.
It is possible to replace them by more descriptive names, that may even contain LaTeX code if you want pretty names in your papers.

In [4]:
name_map = {
    'ace': '$ACE$',
    'choco': '$Choco$',
    'cosoco': '$Cosoco$',
    'sat4j+default': '$Sat4j_{default}$',
}

Based on the map above, we can easily replace the name of the experiment-wares as follows.

In [5]:
analysis = analysis.add_variable(
    new_var='experiment_ware',
    function=lambda xp: name_map[xp['experiment_ware']]
)

## General data preprocessing

The first step is to fix the status that `Cosoco` outputs.
Indeed, where `OPTIMUM FOUND` is expected, `Cosoco` only writes `OPTIMUM`.

In [6]:
analysis = analysis.add_variable(
    new_var='status',
    function=lambda xp: 'OPTIMUM FOUND' if xp['status'] == 'OPTIMUM' else xp['status']
)

Then, to know whether the input problem is a minimization or maximization problem, we must consider the column `cop.way`, but it is only provided by `ACE`.
The function below allows to retrieve this information from the analysis.

In [7]:
def extract_min_or_max(xp):
    """
    Retrieves from the analysis the type of optimization problem corresponding
    to the given input.

    :param xp: The experiment to determine the type of.
    """
    df = analysis.data_frame
    for way in df[df['input'] == xp['input']]['cop.way'].unique():
        if not isnull(way):
            return way
    return None

We can now apply the function above to associate any experiment to its associated objective type.

In [8]:
analysis = analysis.add_variable(
    new_var='objective',
    function=extract_min_or_max
)

To make easier the manipulation of the data related to the experiments, we add a new variable to represent the best value found by the solvers.

In [9]:
def extract_best_value(xp):
    """
    Extracts the best value found by the solver.
    By construction, it is necessarily the latest one.
    
    :param xp: The experiment to determine the best bound of.
    """
    if isinstance(xp['bound_list'], list):
        # Several values were found, the best one is the latest.
        return xp['bound_list'][-1]
    if not isnull(xp['bound_list']):
        # Only one value was found, it is necessarily the best.
        return int(xp['bound_list'])
    # No values were found.
    return None

We now use this function to define a new variable in the analysis.

In [10]:
analysis = analysis.add_variable(
    new_var='best_bound',
    function=extract_best_value
).add_variable(
    new_var='optimum',
    function=lambda xp: xp['best_bound'] if xp['status'] == 'OPTIMUM FOUND' else None
)

We can now check in the data-frame whether all the changes made in this section have been properly applied to the collected data.

In [11]:
analysis.data_frame

,input,experiment_ware,cpu_time,track,status,bound_list,timestamp_list,family,cop.way,timeout,success,user_success,missing,consistent_xp,consistent_input,error,objective,best_bound,optimum
0,Rcpsp_m1_j30_Rcpsp_j30_43_02,$Choco$,1.326810,COP,OPTIMUM FOUND,"[44, 43]","[1.12, 1.12]",Rcpsp,NaN,1200.0,True,True,False,True,True,False,min,43.0,43.0
9745,Rcpsp_m1_j30_Rcpsp_j30_43_02,$Cosoco$,0.033937,None,OPTIMUM FOUND,43,0.000442,Rcpsp,None,1200.0,True,True,False,True,True,False,min,43.0,43.0
5079,Rcpsp_m1_j30_Rcpsp_j30_43_02,$ACE$,2.674830,None,OPTIMUM FOUND,"[55, 51, 50, 43]","[0.41, 0.43, 0.43, 0.44]",Rcpsp,min,1200.0,True,True,False,True,True,False,min,43.0,43.0
1377,Rcpsp_m1_j30_Rcpsp_j30_43_02,$Sat4j_{default}$,240.661000,None,OPTIMUM FOUND,"[143, 131, 129, 116, 115, 108, 107, 106, 105, ...",None,Rcpsp,None,1200.0,True,True,False,True,True,False,min,43.0,43.0
16164,GraphColoring_m1_mono_GraphColoring_2_fullins_5,$Choco$,300.085000,COP,SATISFIABLE,6,5.93,GraphColoring,None,1200.0,True,True,False,True,True,False,min,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18249,PseudoBoolean_opt_factor_Pb_factor_S9_P191_Q389,$Sat4j_{default}$,900.566000,None,UNKNOWN,None,None,PseudoBoolean,None,1200.0,True,True,False,True,True,False,min,NaN,NaN
18402,Rcpsp_m1_j60_Rcpsp_j60_14_07,$Choco$,2.021590,COP,OPTIMUM FOUND,"[314, 313, 312, 72, 71, 69]","[1.76, 1.76, 1.76, 1.76, 1.76, 1.76]",Rcpsp,None,1200.0,True,True,False,True,True,False,min,69.0,69.0
18201,Rcpsp_m1_j60_Rcpsp_j60_14_07,$Cosoco$,0.045985,None,OPTIMUM FOUND,69,0.003777,Rcpsp,None,1200.0,True,True,False,True,True,False,min,69.0,69.0
18582,Rcpsp_m1_j60_Rcpsp_j60_14_07,$ACE$,7.964840,None,OPTIMUM FOUND,"[82, 79, 69]","[1.19, 1.26, 1.34]",Rcpsp,min,1200.0,True,True,False,True,True,False,min,69.0,69.0


## Checking the success and consistency of the results

During our analysis, we will need to know whether a given experiment was successful.
As an example, we provide below the code to check the success of an optimization solver.

In [12]:
def is_success(xp):
    """
    This function checks that a solver either proved the optimality of its best
    bound within the time limit, or proved the input to be unsatisfiable.

    :param xp: The experiment to determine the best bound of.
    """
    return xp['status'] == 'OPTIMUM FOUND' or xp['status'] == 'UNSATISFIABLE'

To make sure that our experiments are consistent, we also need to compare the results obtained by the different experiment-wares.
As an example, we provide below the code to check that if different optimization solvers claim to have found an optimal value, this value must be the same for all solvers.

In [13]:
def is_consistent_by_input(df_input):
    """
    This function checks that the pairwise comparison between two different
    optimal bounds found on the same input is small enough to consider these bounds as consistent.
    """
    # Checking the decision of the solvers.
    decisions = df_input['decision'].unique()
    if 'OPTIMUM FOUND' in decisions and 'UNSATISFIABLE' in decisions:
        # A solver has found an optimal solution while another proved unsatisfiability.
        return False
    if 'SATISFIABLE' in decisions and 'UNSATISFIABLE' in decisions:
        # A solver has found a solution while another proved unsatisfiability.
        return False

    # Checking that at most one optimal value exists.
    best_values_for_complete_search = df_input[df_input['success']]['best_bound'].unique()

    # Checking if the "proved" best bound is less optimal than another non-optimal bound
    if df_input['objective'].unique()[0] == 'min': # in the case of minimization
        best_global_value = df_input['best_bound'].min()
    else: # in the case of maximization
        best_global_value = df_input['best_bound'].max()


    # Checking 
    return best_global_value is None or \
           len(best_values_for_complete_search) <= 1 and \
           best_values_for_complete_search[0] != best_global_value
        

We can now use the functions above to check the consistency of the different experiments in the analysis.

In [14]:
analysis.check_success(is_success)
analysis.check_input_consistency(is_consistent_by_input)

There is no warning raised when checking the consistency of the results, and the table below is empty, so there is apparently no consistency issues with our experiments.

In [15]:
analysis.error_table()

,input,experiment_ware,cpu_time,track,status,bound_list,timestamp_list,family,cop.way,timeout,success,user_success,missing,consistent_xp,consistent_input,error,objective,best_bound,optimum


## Summary and export of the analysis

We can now give a summary of the analysis, that we obtain through the following table.

In [16]:
analysis.description_table()

,analysis
n_experiment_wares,4
n_inputs,4772
n_experiments,19088
n_missing_xp,0
n_inconsistent_xp,0
n_inconsistent_xp_due_to_input,0
more_info_about_variables,<analysis>.data_frame.describe(include='all')


Finally, the analysis is exported, both to share the data to allow the reproducibility of the analysis, and to reuse it in other notebooks dedicated to more specific analyses.

In [17]:
analysis.export('.cache')